In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv("datasets/fuga_clientes_empresa_telefonica_construccion.csv")

In [ ]:
df[:5]

In [ ]:
df.shape

In [ ]:
df.describe(include='all')

In [ ]:
df.columns

In [ ]:
df_num = pd.get_dummies(df.drop('Customer ID', axis=1))

In [ ]:
df_num.insert(loc=0, value=df['Customer ID'], column='Customer ID')

In [ ]:
df_num.shape

In [ ]:
df_num[:3]

In [ ]:
df_num.columns

In [ ]:
df_num.columns

In [ ]:
#plt.figure(figsize=(50,15))
for var in df_num.columns[1:]:
    df_num[var].hist()
    plt.title(var)
    plt.show()

In [ ]:
df_num.isnull().sum()

In [ ]:
print(df_num.columns)

In [ ]:
df_num_copia = df_num.copy()
y = df_num_copia["Churn Status"].values
df_num_copia = df_num_copia.drop("Churn Status", axis=1)
X = df_num_copia.values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,
                                                    random_state=1)

In [ ]:
X_train[0]

In [ ]:
IDs_train = X_train[:,0]
IDs_test = X_test[:,0]
X_train = X_train[:,1:]
X_test = X_test[:,1:]

In [ ]:
X_train[0]

In [ ]:
X_train = X_train.astype('float')
X_test = X_test.astype('float')

In [ ]:
X_train.var(axis=0)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [ ]:
X_train_std.var(axis=0)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

In [ ]:
X_train_std.shape, X_train_pca.shape

In [ ]:
np.unique(y_train)

In [ ]:
comp_horiz=1
comp_vert=2
plt.figure(figsize=(7,7))
plt.plot(X_train_pca[y_train==0,comp_horiz],
         X_train_pca[y_train==0,comp_vert], '.', label="Churn=0")
plt.plot(X_train_pca[y_train==1,comp_horiz],
         X_train_pca[y_train==1,comp_vert], '.', label="Churn=1")
plt.xlabel('principal component '+str(comp_horiz))
plt.ylabel('principal component '+str(comp_vert))
plt.legend();

In [ ]:
df_num_copia.columns

## Dummy classifier

In [ ]:
from sklearn.dummy import DummyClassifier

clf_dummy = DummyClassifier(strategy='most_frequent')
clf_dummy.fit(X_train,y_train)
clf_dummy.score(X_train,y_train), clf_dummy.score(X_test,y_test)

### Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from libreria_aux_arboles import tree_to_code

In [ ]:
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(X_train, y_train)

In [ ]:
tree_to_code(clf, df_num_copia.columns[1:])

In [ ]:
clf.score(X_train, y_train), clf.score(X_test, y_test)

### Grid search in Decision Trees using Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(estimator=DecisionTreeClassifier(criterion='gini'),
                  cv=5, param_grid={'max_leaf_nodes':range(2,40+1),
#                                    'max_depth':[1,2,3,4]
                                   },
                  refit=True)
gs.fit(X_train, y_train)

In [ ]:
gs.best_estimator_

In [ ]:
gs.score(X_train, y_train), gs.score(X_test, y_test)

In [ ]:
gs.predict_proba(X_test[:10])

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf.score(X_train, y_train), clf.score(X_test, y_test)

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=51,
                             max_depth=10)
clf.fit(X_train, y_train)
clf.score(X_train, y_train), clf.score(X_test, y_test)